In [38]:
import os
import re
from collections import Counter
import subprocess

import pandas as pd

import mgitools.os_helpers as os_helpers

## bammap creation

In [32]:
bam_dir = '../bams/'
bam_dir = '/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/'
output_bammap = '../TCGA-TGCT.bammap'

In [33]:
metadata = pd.read_csv('/gscmnt/gc2686/rna_editing/sandbox/gdc_sample_sheet.2020-06-20.tsv', sep='\t')
metadata = metadata.set_index('File ID')
metadata

,File Name,Data Category,Data Type,Project ID,Case ID,Sample ID,Sample Type
File ID,,,,,,,
753dbfe7-f821-4331-a938-936c90f4cd30,TCGA-2G-AALY-01A-11D-A42Y-10_Illumina_gdc_real...,Sequencing Reads,Aligned Reads,TCGA-TGCT,TCGA-2G-AALY,TCGA-2G-AALY-01A,Primary Tumor
f590c324-179c-45d5-a710-fc45e3174390,TCGA-X3-A8G4-01A-11D-A435-10_Illumina_gdc_real...,Sequencing Reads,Aligned Reads,TCGA-TGCT,TCGA-X3-A8G4,TCGA-X3-A8G4-01A,Primary Tumor
6affbdbc-1488-4cc2-9104-572b9c59fd00,TCGA-ZM-AA0B-10A-01D-A438-10_Illumina_gdc_real...,Sequencing Reads,Aligned Reads,TCGA-TGCT,TCGA-ZM-AA0B,TCGA-ZM-AA0B-10A,Blood Derived Normal
4c906845-8b21-4e54-8bf2-32dc763a8ebc,TCGA-2G-AAKO-01A-11D-A42Y-10_Illumina_gdc_real...,Sequencing Reads,Aligned Reads,TCGA-TGCT,TCGA-2G-AAKO,TCGA-2G-AAKO-01A,Primary Tumor
c5666f04-0776-44e0-9d04-7a78c9be3f71,TCGA-XE-AAOC-10A-01D-A438-10_Illumina_gdc_real...,Sequencing Reads,Aligned Reads,TCGA-TGCT,TCGA-XE-AAOC,TCGA-XE-AAOC-10A,Blood Derived Normal
f1ed2f79-d423-4935-ba98-3a92f762f6c5,TCGA-2G-AAEX-10A-01D-A433-10_Illumina_gdc_real...,Sequencing Reads,Aligned Reads,TCGA-TGCT,TCGA-2G-AAEX,TCGA-2G-AAEX-10A,Blood Derived Normal
57ff6694-facf-4860-ae5b-06a84ef62b67,TCGA-ZM-AA06-10A-01D-A438-10_Illumina_gdc_real...,Sequencing Reads,Aligned Reads,TCGA-TGCT,TCGA-ZM-AA06,TCGA-ZM-AA06-10A,Blood Derived Normal
9a485b68-42f3-4cbf-a008-dfd89671086f,TCGA-XE-AAOJ-10A-01D-A438-10_Illumina_gdc_real...,Sequencing Reads,Aligned Reads,TCGA-TGCT,TCGA-XE-AAOJ,TCGA-XE-AAOJ-10A,Blood Derived Normal
0d3c383d-d433-4b57-b1b7-12a1e714a486,06b7c5b0-b891-4cd9-a54d-bd0dbe8f3ff9_wxs_gdc_r...,Sequencing Reads,Aligned Reads,TCGA-TGCT,TCGA-2G-AAI4,TCGA-2G-AAI4-01A,Primary Tumor


In [34]:
Counter(metadata['Sample Type']).most_common()

[('Primary Tumor', 263),
 ('Blood Derived Normal', 263),
 ('Additional - New Primary', 7)]

In [35]:
fps = sorted(os_helpers.listfiles(bam_dir, regex='.bam$'))
len(fps), fps[:5]

(341,
 ['/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/02645a88-2b1b-4390-8751-8d11681b138a/5a0b350a-b4b1-499f-b7cd-a40923b6534d_wxs_gdc_realn.bam',
  '/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/02a7a293-24ab-42e7-b7ef-4ee53c76da8c/ce9e2239-0113-47f6-a42e-af4e40a1323e_wxs_gdc_realn.bam',
  '/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/02a83d97-ae6a-428b-92bd-83a428475e92/TCGA-2G-AAFG-05A-11D-A42Y-10_Illumina_gdc_realn.bam',
  '/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/03451dc0-4fd1-4718-a1c1-4b4bfa5efeac/TCGA-SN-A84Y-10A-01D-A438-10_Illumina_gdc_realn.bam',
  '/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/070ad00f-9136-4ada-950f-ede6bc641f15/TCGA-2G-AAFG-10A-01D-A433-10_Illumina_gdc_realn.bam'])

In [10]:
# sample_name	case	disease	experimental_strategy	sample_type	data_path	filesize	data_format	reference	UUID	system
11LU013.WGS.N.hg38	11LU013	LUAD	WGS	blood_normal	/gscmnt/gc2619/dinglab_cptac3/GDC_import/data/89783e46-3269-42ad-8b4a-6b3de3fd47dc/e04e7004-0cc8-42d0-9276-a290b24940ce_gdc_realn.bam	96565337883	BAM	hg38	89783e46-3269-42ad-8b4a-6b3de3fd47dc	MGI




SyntaxError: invalid syntax (<ipython-input-10-6eb38b9bb5ba>, line 2)

In [36]:
d = {
    '# sample_name': [],
    'case': [],
    'disease': [],
    'experimental_strategy': [],
    'sample_type': [],
    'data_path': [],
    'filesize': [],
    'data_format': [],
    'reference': [],
    'UUID': [],
    'system': []
}
for fp in fps:
    pieces = fp.split('/')
    uuid = pieces[-2]
    case = metadata.loc[uuid, 'Case ID']
    if metadata.loc[uuid, 'Sample Type'] in ['Blood Derived Normal', 'Primary Tumor']:
        sample_type = 'blood_normal' if metadata.loc[uuid, 'Sample Type'] == 'Blood Derived Normal' else 'tumor'
        filesize = os.path.getsize(fp)
        character = 'N' if sample_type == 'blood_normal' else 'T'
        sample_name = f'{case}.WXS.{character}.hg38'

        d['# sample_name'].append(sample_name)
        d['case'].append(case)
        d['disease'].append('TGCT')
        d['experimental_strategy'].append('WXS')
        d['sample_type'].append(sample_type)
        d['data_path'].append(fp)
        d['filesize'].append(filesize)
        d['data_format'].append('BAM')
        d['reference'].append('hg38')
        d['UUID'].append(uuid)
        d['system'].append('MGI')
bammap = pd.DataFrame.from_dict(d)
bammap
    
    

,# sample_name,case,disease,experimental_strategy,sample_type,data_path,filesize,data_format,reference,UUID,system
0,TCGA-XE-AAO7.WXS.T.hg38,TCGA-XE-AAO7,TGCT,WXS,tumor,/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/0264...,62559578473,BAM,hg38,02645a88-2b1b-4390-8751-8d11681b138a,MGI
1,TCGA-YU-A90R.WXS.T.hg38,TCGA-YU-A90R,TGCT,WXS,tumor,/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/02a7...,44291562018,BAM,hg38,02a7a293-24ab-42e7-b7ef-4ee53c76da8c,MGI
2,TCGA-SN-A84Y.WXS.N.hg38,TCGA-SN-A84Y,TGCT,WXS,blood_normal,/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/0345...,17000519818,BAM,hg38,03451dc0-4fd1-4718-a1c1-4b4bfa5efeac,MGI
3,TCGA-2G-AAFG.WXS.N.hg38,TCGA-2G-AAFG,TGCT,WXS,blood_normal,/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/070a...,12729573440,BAM,hg38,070ad00f-9136-4ada-950f-ede6bc641f15,MGI
4,TCGA-2G-AAGO.WXS.N.hg38,TCGA-2G-AAGO,TGCT,WXS,blood_normal,/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/07d8...,13320338881,BAM,hg38,07d849ad-90b8-4067-8b45-e62a72514df5,MGI
5,TCGA-XE-A8H5.WXS.N.hg38,TCGA-XE-A8H5,TGCT,WXS,blood_normal,/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/07e9...,14423250406,BAM,hg38,07e9af59-1016-45f3-bc5b-7d6c09c7624f,MGI
6,TCGA-XE-AAOB.WXS.N.hg38,TCGA-XE-AAOB,TGCT,WXS,blood_normal,/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/08cc...,15353296172,BAM,hg38,08cc4424-aa6d-4f6e-93d5-02d7a257c532,MGI
7,TCGA-2G-AAF0.WXS.T.hg38,TCGA-2G-AAF0,TGCT,WXS,tumor,/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/095d...,41723582182,BAM,hg38,095d0d8f-cddf-47b1-ac22-1c75d7ec6870,MGI
8,TCGA-YU-A90S.WXS.N.hg38,TCGA-YU-A90S,TGCT,WXS,blood_normal,/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/0a2a...,17097528382,BAM,hg38,0a2a49d5-0a15-4378-ab1b-05495f4e015b,MGI
9,TCGA-2G-AAIE.WXS.N.hg38,TCGA-2G-AAIE,TGCT,WXS,blood_normal,/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/0ac8...,39055441574,BAM,hg38,0ac8fb26-062f-44f3-9333-7699fe645c05,MGI


In [37]:
bammap.to_csv(output_bammap, sep='\t', index=False, header=True)